In [1]:
#pip install pyarrow

In [2]:
# Импортируем библиотеки
import pandas as pd
import seaborn as sns
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import make_scorer, roc_auc_score

H:\Anaconda\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Отбор значимых признаков

### Часть 1

In [3]:
# Считываем названия фичей, преобразуем в датафрейм
columns = pd.read_json('features_types.json', typ='series')
data_columns = pd.DataFrame(columns).reset_index()
data_columns.rename(columns = {'index': 'column', 0: 'type'}, 
                    inplace = True)

In [4]:
# Формируем список фичей для выгрузки из панельки
list_columns = data_columns[:900].column.tolist()
list_columns.insert(0, 'target')
list_columns.insert(1, 'period')

In [5]:
# Выгружаем данные
data = pd.read_parquet('dataset_train.parquet', engine='pyarrow', columns=list_columns)

In [6]:
# Считаем фичи с долей пропусков более 50%
list_feat = []
for column in data_columns[0:900].column.tolist():
    if data[column].isna().sum() / data[column].isna().count() > 0.5:
        list_feat.append(column)
len(list_feat)

35

In [7]:
# Формируем список без фичей с пропусками
list_wo_miss = list(set(list_columns) - set(list_feat))
len(list_wo_miss)

867

In [8]:
# Переопределяем датафрейм с фичами для выгрузки
data_columns = data_columns.loc[data_columns['column'].isin(list_wo_miss)]

# Разбиваем полученный датасет на фичи по типу
data_columns_numeric = data_columns.loc[data_columns['type'] == 'numeric']

data_columns_cat = data_columns.loc[(data_columns['type'] == 'categorical_int') | 
                                    (data_columns['type'] == 'categorical_str')]

In [9]:
# Формируем список фичей для выгрузки из панельки
list_columns_numeric = data_columns_numeric.column.tolist()
list_columns_numeric.insert(0, 'target')
list_columns_numeric.insert(1, 'period')

list_columns_cat = data_columns_cat.column.tolist()

In [10]:
data_cat = pd.read_parquet('dataset_train.parquet', engine='pyarrow', columns=list_columns_cat)
data_cat.isna().sum().max()
# пропусков нет 

0

In [11]:
data_numeric = pd.read_parquet('dataset_train.parquet', engine='pyarrow', columns=list_columns_numeric)

In [12]:
for column in list_columns_numeric:
    data_numeric[column].fillna(data_numeric[column].median(), inplace = True)
data_numeric.isna().sum().max()

0

In [13]:
# Датафрейм с заполненными пропусками
data_fill = data_numeric.join(data_cat)
data_numeric = []
data_cat = []

#### Отбор значимых признаков

In [14]:
# Задаем объясняемую и объясняющие переменные
features = data_fill.drop(columns=['target', 'period'])
target = data_fill['target']

In [15]:
# Оределеяем константу для random_state
STATE = 442

# Создаем метрику качества для GridSearch
roc_auc_scorer = make_scorer(roc_auc_score, greater_is_better=True,
                             needs_threshold=True)


# Определяем модель
model = lgb.LGBMClassifier(random_state = STATE)

# Задаем параметры для GridSearchCV
params = {
     'max_depth': [10],
     'reg_alpha':[0.01],
     'num_leaves': [5]}

In [16]:
%%time
# Настраиваем GridSearch
grid_lgbm = GridSearchCV(model, 
                         params, 
                         cv = 3, 
                         scoring = roc_auc_scorer, 
                         n_jobs = -1)

# Находим модель с лучшим значением метрики
grid_lgbm.fit(features[features.columns.tolist()], target)


print(f'ROC-AUC для лучшей модели бустинга составило: '
      f'{-grid_lgbm.best_score_:.6}')
print(f'Гиперпараметры лучшей модели: {grid_lgbm.best_params_}')

ROC-AUC для лучшей модели бустинга составило: -0.694355
Гиперпараметры лучшей модели: {'max_depth': 10, 'num_leaves': 5, 'reg_alpha': 0.01}
Wall time: 31min 37s


In [17]:
# Достаем значимость объясняющих переменных
importances = grid_lgbm.best_estimator_.feature_importances_

# Создаем датафрейм
data_tuples = list(zip(features.columns.tolist(), importances.tolist()))
feat_imp_df = pd.DataFrame(data_tuples, columns=['feature','importance'])

In [18]:
# Формируем список объясняющих переменных с ненулевой значимостью
list_columns = feat_imp_df.feature.tolist()
list_feat = feat_imp_df.query('importance == 0').feature.tolist()
list_feat_imp = list(set(list_columns) - set(list_feat))

In [19]:
# Переопределяем набор объясняющих переменных для модели
features = features[list_feat_imp]

In [20]:
%%time

# Переобучаем модель
grid_lgbm.fit(features[features.columns.tolist()], target)

print(f'ROC-AUC для лучшей модели бустинга составило: '
      f'{-grid_lgbm.best_score_:.6}')

ROC-AUC для лучшей модели бустинга составило: -0.698232
Wall time: 3min 11s


In [21]:
# Формируем список фичей с ненулевой значимостью
importances = grid_lgbm.best_estimator_.feature_importances_
data_tuples = list(zip(features.columns.tolist(), importances.tolist()))
feat_imp_df = pd.DataFrame(data_tuples, columns=['feature','importance'])
feat_imp_df.sort_values(by = 'importance', ascending = False)

,feature,importance
81,communication_availability_3_1_flg,17
142,markers_706_1_cnt,16
23,materials_details_0_1_ctg,15
58,markers_60_1_cnt,12
147,user_devices_0_1_cnt,10
...,...,...
57,markers_663_1_cnt,1
55,payments_details_15_1_cnt,1
54,payments_details_5_6_cnt,1
53,charges_details_24_1_sum,1


In [22]:
# Экспортируем файл
feat_imp_df.to_csv (r'C:\Users\anato\Desktop\ds\Хакатон R1\1st_part_features.csv', index= False )

### Часть 2

In [23]:
# Считываем названия фичей, преобразуем в датафрейм
columns = pd.read_json('features_types.json', typ='series')
data_columns = pd.DataFrame(columns).reset_index()
data_columns.rename(columns = {'index': 'column', 0: 'type'}, 
                    inplace = True)

In [24]:
# Формируем список фичей для выгрузки из панельки
list_columns = data_columns[900:1800].column.tolist()
list_columns.insert(0, 'target')
list_columns.insert(1, 'period')

In [25]:
# Выгружаем данные
data = pd.read_parquet('dataset_train.parquet', engine='pyarrow', columns=list_columns)

In [26]:
# Считаем фичи с долей пропусков более 50%
list_feat = []
for column in data_columns[900:1800].column.tolist():
    if data[column].isna().sum() / data[column].isna().count() > 0.5:
        list_feat.append(column)
len(list_feat)

174

In [27]:
# Формируем список без фичей с пропусками
list_wo_miss = list(set(list_columns) - set(list_feat))
len(list_wo_miss)

728

In [28]:
# Переопределяем датафрейм с фичами для выгрузки
data_columns = data_columns.loc[data_columns['column'].isin(list_wo_miss)]

# Разбиваем полученный датасет на фичи по типу
data_columns_numeric = data_columns.loc[data_columns['type'] == 'numeric']

data_columns_cat = data_columns.loc[(data_columns['type'] == 'categorical_int') | 
                                    (data_columns['type'] == 'categorical_str')]

In [29]:
# Формируем список фичей для выгрузки из панельки
list_columns_numeric = data_columns_numeric.column.tolist()
list_columns_numeric.insert(0, 'target')
list_columns_numeric.insert(1, 'period')

list_columns_cat = data_columns_cat.column.tolist()

In [30]:
data_cat = pd.read_parquet('dataset_train.parquet', engine='pyarrow', columns=list_columns_cat)
data_cat.isna().sum().max()
# пропусков нет 

0

In [31]:
data_numeric = pd.read_parquet('dataset_train.parquet', engine='pyarrow', columns=list_columns_numeric)

In [32]:
for column in list_columns_numeric:
    data_numeric[column].fillna(data_numeric[column].median(), inplace = True)
data_numeric.isna().sum().max()

0

In [33]:
# Датафрейм с заполненными пропусками
data_fill = data_numeric.join(data_cat)
data_numeric = []
data_cat = []

#### Отбор значимых признаков

In [34]:
# Задаем объясняемую и объясняющие переменные
features = data_fill.drop(columns=['target', 'period'])
target = data_fill['target']

In [35]:
# Оределеяем константу для random_state
STATE = 442

# Создаем метрику качества для GridSearch
roc_auc_scorer = make_scorer(roc_auc_score, greater_is_better=True,
                             needs_threshold=True)


# Определяем модель
model = lgb.LGBMClassifier(random_state = STATE)

# Задаем параметры для GridSearchCV
params = {
     'max_depth': [10],
     'reg_alpha':[0.01],
     'num_leaves': [5]}

In [36]:
%%time
# Настраиваем GridSearch
grid_lgbm = GridSearchCV(model, 
                         params, 
                         cv = 3, 
                         scoring = roc_auc_scorer, 
                         n_jobs = -1)

# Находим модель с лучшим значением метрики
grid_lgbm.fit(features[features.columns.tolist()], target)


print(f'ROC-AUC для лучшей модели бустинга составило: '
      f'{-grid_lgbm.best_score_:.6}')
print(f'Гиперпараметры лучшей модели: {grid_lgbm.best_params_}')

ROC-AUC для лучшей модели бустинга составило: -0.704455
Гиперпараметры лучшей модели: {'max_depth': 10, 'num_leaves': 5, 'reg_alpha': 0.01}
Wall time: 14min 30s


In [37]:
# Достаем значимость объясняющих переменных
importances = grid_lgbm.best_estimator_.feature_importances_

# Создаем датафрейм
data_tuples = list(zip(features.columns.tolist(), importances.tolist()))
feat_imp_df = pd.DataFrame(data_tuples, columns=['feature','importance'])

In [38]:
# Формируем список объясняющих переменных с ненулевой значимостью
list_columns = feat_imp_df.feature.tolist()
list_feat = feat_imp_df.query('importance == 0').feature.tolist()
list_feat_imp = list(set(list_columns) - set(list_feat))

In [39]:
# Переопределяем набор объясняющих переменных для модели
features = features[list_feat_imp]

In [40]:
%%time

# Переобучаем модель
grid_lgbm.fit(features[features.columns.tolist()], target)

print(f'ROC-AUC для лучшей модели бустинга составило: '
      f'{-grid_lgbm.best_score_:.6}')

ROC-AUC для лучшей модели бустинга составило: -0.705403
Wall time: 29 s


In [41]:
# Формируем список фичей с ненулевой значимостью
importances = grid_lgbm.best_estimator_.feature_importances_
data_tuples = list(zip(features.columns.tolist(), importances.tolist()))
feat_imp_df = pd.DataFrame(data_tuples, columns=['feature','importance'])
feat_imp_df.sort_values(by = 'importance', ascending = False)

,feature,importance
2,communication_availability_40_1_ctg,16
81,tariff_plans_21_1_max,14
4,traffic_details_62_1_sum,14
126,materials_details_12_1_flg,13
92,spas_symptoms_agr_79_6_sum,12
...,...,...
66,traffic_details_1_3_cnt,1
62,traffic_details_42_1_avg,1
59,spas_symptoms_agr_38_3_avg,1
56,payments_details_47_3_avg,1


In [42]:
# Экспортируем файл
feat_imp_df.to_csv (r'C:\Users\anato\Desktop\ds\Хакатон R1\2nd_part_features.csv', index= False )

### Часть 3

In [101]:
# Формируем список фичей для выгрузки из панельки
list_columns = data_columns[1800:].column.tolist()
list_columns.insert(0, 'target')
list_columns.insert(1, 'period')

In [102]:
# Выгружаем данные
data = pd.read_parquet('dataset_train.parquet', engine='pyarrow', columns=list_columns)

In [105]:
# Баланс классов
data.target.sum() / data.target.count()

0.0077896440037260395

In [106]:
# Считаем фичи с долей пропусков более 50%
list_feat = []
for column in data_columns[1800:].column.tolist():
    if data[column].isna().sum() / data[column].isna().count() > 0.5:
        list_feat.append(column)
len(list_feat)

334

In [107]:
# Формируем список без фичей с пропусками
list_wo_miss = list(set(list_columns) - set(list_feat))
len(list_wo_miss)

644

In [108]:
# Переопределяем датафрейм с фичами для выгрузки
data_columns = data_columns.loc[data_columns['column'].isin(list_wo_miss)]

# Разбиваем полученный датасет на фичи по типу
data_columns_numeric = data_columns.loc[data_columns['type'] == 'numeric']

data_columns_cat = data_columns.loc[(data_columns['type'] == 'categorical_int') | 
                                    (data_columns['type'] == 'categorical_str')]

In [109]:
# Формируем список фичей для выгрузки из панельки
list_columns_numeric = data_columns_numeric.column.tolist()
list_columns_numeric.insert(0, 'target')
list_columns_numeric.insert(1, 'period')

list_columns_cat = data_columns_cat.column.tolist()

In [110]:
data_cat = pd.read_parquet('dataset_train.parquet', engine='pyarrow', columns=list_columns_cat)
data_cat.isna().sum().max()
# пропусков нет 

0

In [111]:
data_numeric = pd.read_parquet('dataset_train.parquet', engine='pyarrow', columns=list_columns_numeric)

In [112]:
for column in list_columns_numeric:
    data_numeric[column].fillna(data_numeric[column].median(), inplace = True)
data_numeric.isna().sum().max()

0

In [113]:
# Датафрейм с заполненными пропусками
data_fill = data_numeric.join(data_cat)
data_numeric = []
data_cat = []

#### Отбор значимых признаков

In [114]:
# Задаем объясняемую и объясняющие переменные
features = data_fill.drop(columns=['target', 'period'])
target = data_fill['target']

In [115]:
# Оределеяем константу для random_state
STATE = 442

# Создаем метрику качества для GridSearch
roc_auc_scorer = make_scorer(roc_auc_score, greater_is_better=True,
                             needs_threshold=True)


# Определяем модель
model = lgb.LGBMClassifier(random_state = STATE)

# Задаем параметры для GridSearchCV
params = {
     'max_depth': [10],
     'reg_alpha':[0.01],
     'num_leaves': [5]}

In [116]:
%%time
# Настраиваем GridSearch
grid_lgbm = GridSearchCV(model, 
                         params, 
                         cv = 3, 
                         scoring = roc_auc_scorer, 
                         n_jobs = -1)

# Находим модель с лучшим значением метрики
grid_lgbm.fit(features[features.columns.tolist()], target)


print(f'ROC-AUC для лучшей модели бустинга составило: '
      f'{-grid_lgbm.best_score_:.6}')
print(f'Гиперпараметры лучшей модели: {grid_lgbm.best_params_}')

ROC-AUC для лучшей модели бустинга составило: -0.695905
Гиперпараметры лучшей модели: {'max_depth': 10, 'num_leaves': 5, 'reg_alpha': 0.01}
Wall time: 9min 19s
Parser   : 131 ms


In [117]:
# Достаем значимость объясняющих переменных
importances = grid_lgbm.best_estimator_.feature_importances_

# Создаем датафрейм
data_tuples = list(zip(features.columns.tolist(), importances.tolist()))
feat_imp_df = pd.DataFrame(data_tuples, columns=['feature','importance'])

In [118]:
# Формируем список объясняющих переменных с ненулевой значимостью
list_columns = feat_imp_df.feature.tolist()
list_feat = feat_imp_df.query('importance == 0').feature.tolist()
list_feat_imp = list(set(list_columns) - set(list_feat))

In [119]:
# Переопределяем набор объясняющих переменных для модели
features = features[list_feat_imp]

In [120]:
%%time

# Переобучаем модель
grid_lgbm.fit(features[features.columns.tolist()], target)

print(f'ROC-AUC для лучшей модели бустинга составило: '
      f'{-grid_lgbm.best_score_:.6}')

ROC-AUC для лучшей модели бустинга составило: -0.697521
Wall time: 30.1 s


In [121]:
# Формируем список фичей с ненулевой значимостью
importances = grid_lgbm.best_estimator_.feature_importances_
data_tuples = list(zip(features.columns.tolist(), importances.tolist()))
feat_imp_df = pd.DataFrame(data_tuples, columns=['feature','importance'])
feat_imp_df.sort_values(by = 'importance', ascending = False)

,feature,importance
43,type_av_100_0_1_ctg,19
53,basic_info_0_0_avg,13
72,materials_details_13_1_ctg,12
60,user_devices_30_1_cnt,10
78,campaigns_250_1_sum,10
...,...,...
46,markers_830_1_cnt,0
50,spas_symptoms_agr_286_12_sum,0
62,markers_852_1_cnt,0
73,campaigns_392_3_part,0


In [122]:
# Формируем список объясняющих переменных с ненулевой значимостью
list_columns = feat_imp_df.feature.tolist()
list_feat = feat_imp_df.query('importance == 0').feature.tolist()
list_feat_imp = list(set(list_columns) - set(list_feat))

In [123]:
# Переопределяем набор объясняющих переменных для модели
features = features[list_feat_imp]

In [124]:
%%time

# Переобучаем модель
grid_lgbm.fit(features[features.columns.tolist()], target)

print(f'ROC-AUC для лучшей модели бустинга составило: '
      f'{-grid_lgbm.best_score_:.6}')

ROC-AUC для лучшей модели бустинга составило: -0.696898
Wall time: 1min 5s


In [125]:
# Формируем список фичей с ненулевой значимостью
importances = grid_lgbm.best_estimator_.feature_importances_
data_tuples = list(zip(features.columns.tolist(), importances.tolist()))
feat_imp_df = pd.DataFrame(data_tuples, columns=['feature','importance'])
feat_imp_df.sort_values(by = 'importance', ascending = False)

,feature,importance
43,type_av_100_0_1_ctg,19
51,basic_info_0_0_avg,13
68,materials_details_13_1_ctg,12
58,user_devices_30_1_cnt,10
73,campaigns_250_1_sum,10
...,...,...
93,campaigns_401_3_part,1
25,campaigns_386_6_cnt,1
24,user_active_18_1_flg,1
96,campaigns_402_3_sum,1


In [126]:
# Экспортируем файл
feat_imp_df.to_csv (r'C:\Users\anato\Desktop\ds\Хакатон R1\3rd_part_features.csv', index= False )

## Отбор итоговых признаков

In [127]:
column_first = pd.read_csv('1st_part_features.csv')
column_second = pd.read_csv('2nd_part_features.csv')
column_third = pd.read_csv('3rd_part_features.csv')

In [128]:
column_first = column_first.feature.tolist()
column_second = column_second.feature.tolist()
column_third = column_third.feature.tolist()

In [129]:
columns_imp = column_first + column_second + column_third
#column_first, column_second, column_third = [], [], []

In [130]:
# Считываем названия фичей, преобразуем в датафрейм
columns = pd.read_json('features_types.json', typ='series')
data_columns = pd.DataFrame(columns).reset_index()
data_columns.rename(columns = {'index': 'column', 0: 'type'}, 
                    inplace = True)

#test = pd.read_parquet('features_oot.parquet', engine='pyarrow')

In [131]:
data_columns

,column,type
0,markers_0_1_cnt,numeric
1,markers_1_1_cnt,numeric
2,markers_2_1_cnt,numeric
3,markers_3_1_cnt,numeric
4,markers_4_1_cnt,numeric
...,...,...
2771,markers_941_1_cnt,numeric
2772,markers_942_1_cnt,numeric
2773,markers_943_1_cnt,numeric
2774,markers_944_1_cnt,numeric


In [132]:
data_columns_imp = pd.DataFrame(columns_imp)
data_columns_imp.rename(columns = {0: 'column'}, 
                    inplace = True)

In [133]:
data_columns_imp = data_columns_imp.merge(data_columns, on = 'column')
columns_imp.insert(0, 'target')
columns_imp.insert(1, 'period')

In [134]:
# Выгружаем данные
data = pd.read_parquet('dataset_train.parquet', engine='pyarrow', columns=columns_imp)

In [135]:
# Разбиваем полученный датасет на фичи по типу
data_columns_numeric = data_columns_imp.loc[data_columns_imp['type'] == 'numeric']

data_columns_cat = data_columns_imp.loc[(data_columns_imp['type'] == 'categorical_int') | 
                                        (data_columns_imp['type'] == 'categorical_str')]

In [136]:
# Формируем список фичей для выгрузки из панельки
list_columns_numeric = data_columns_numeric.column.tolist()
list_columns_numeric.insert(0, 'target')
list_columns_numeric.insert(1, 'period')

list_columns_cat = data_columns_cat.column.tolist()

In [137]:
data_cat = pd.read_parquet('dataset_train.parquet', engine='pyarrow', columns=list_columns_cat)
data_cat.isna().sum().max()
# пропусков нет 

0

In [138]:
data_numeric = pd.read_parquet('dataset_train.parquet', engine='pyarrow', columns=list_columns_numeric)

In [139]:
for column in list_columns_numeric:
    data_numeric[column].fillna(data_numeric[column].median(), inplace = True)
data_numeric.isna().sum().max()

0

In [140]:
# Датафрейм с заполненными пропусками
data_fill = data_numeric.join(data_cat)
data_numeric = []
data_cat = []

# Предобработка признаков для обучения модели

In [141]:
columns_imp = pd.read_csv('full_features_with_drop.csv')
columns_imp = columns_imp.feature.tolist()

In [142]:
# Считываем названия фичей, преобразуем в датафрейм
columns = pd.read_json('features_types.json', typ='series')
data_columns = pd.DataFrame(columns).reset_index()
data_columns.rename(columns = {'index': 'column', 0: 'type'}, 
                    inplace = True)

#test = pd.read_parquet('features_oot.parquet', engine='pyarrow')

In [143]:
data_columns = pd.DataFrame(columns).reset_index()
data_columns.rename(columns = {'index': 'column', 0: 'type'}, 
                    inplace = True)

In [144]:
data_columns_imp = pd.DataFrame(columns_imp)
data_columns_imp.rename(columns = {0: 'column'}, 
                    inplace = True)

In [145]:
data_columns_imp = data_columns_imp.merge(data_columns, on = 'column')
columns_imp.insert(0, 'target')
columns_imp.insert(1, 'period')

In [146]:
# Выгружаем данные
data = pd.read_parquet('dataset_train.parquet', engine='pyarrow', columns=columns_imp)

In [147]:
# Разбиваем полученный датасет на фичи по типу
data_columns_numeric = data_columns_imp.loc[data_columns_imp['type'] == 'numeric']

data_columns_cat = data_columns_imp.loc[(data_columns_imp['type'] == 'categorical_int') | 
                                        (data_columns_imp['type'] == 'categorical_str')]

In [148]:
# Формируем список фичей для выгрузки из панельки
list_columns_numeric = data_columns_numeric.column.tolist()
list_columns_numeric.insert(0, 'target')
list_columns_numeric.insert(1, 'period')

list_columns_cat = data_columns_cat.column.tolist()

In [149]:
data_cat = pd.read_parquet('dataset_train.parquet', engine='pyarrow', columns=list_columns_cat)
data_cat.isna().sum().max()
# пропусков нет 

0

In [150]:
data_numeric = pd.read_parquet('dataset_train.parquet', engine='pyarrow', columns=list_columns_numeric)

In [151]:
for column in list_columns_numeric:
    data_numeric[column].fillna(data_numeric[column].median(), inplace = True)
data_numeric.isna().sum().max()

0

In [152]:
# Датафрейм с заполненными пропусками
data_fill = data_numeric.join(data_cat)
data_numeric = []
data_cat = []

In [153]:
data_fill

,target,period,markers_706_1_cnt,spas_symptoms_agr_222_6_std,markers_476_1_cnt,user_active_24_0_dt,markers_743_1_cnt,markers_237_1_cnt,markers_558_1_cnt,markers_757_1_cnt,...,tariff_plans_23_1_ctg,communication_availability_40_1_ctg,tariff_plans_14_1_ctg,communication_availability_35_1_ctg,communication_availability_47_1_flg,communication_availability_48_1_flg,communication_availability_17_1_flg,tariff_plans_16_1_ctg,communication_availability_67_1_flg,user_lifetime_0_1_ctg
0,0,202208,-0.357336,-0.120629,-0.764699,1.544611,-0.055048,-0.18704,-0.03591,-0.040255,...,310,3,28,3,0,0,0,423,0,0
1,0,202210,-0.357336,-0.120629,-0.091028,1.598230,-0.055048,-0.18704,-0.03591,-0.040255,...,310,2,28,3,0,0,1,423,0,0
2,1,202210,-0.357336,-0.120629,-0.764699,1.598230,-0.055048,-0.18704,-0.03591,-0.040255,...,310,0,28,3,0,0,0,423,0,0
3,0,202212,-0.357336,-0.120629,0.871360,1.651849,-0.055048,-0.18704,-0.03591,-0.040255,...,287,5,28,3,0,0,0,423,0,0
4,0,202211,-0.357336,-0.120629,-0.091028,1.625479,-0.055048,-0.18704,-0.03591,-0.040255,...,320,5,28,3,0,0,0,423,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702081,0,202209,-0.357336,-0.120629,1.833747,-1.565282,-0.055048,-0.18704,-0.03591,-0.040255,...,215,5,28,3,0,0,1,423,0,13
702082,0,202201,-0.357336,-0.120629,-0.764699,-0.473567,-0.055048,-0.18704,-0.03591,-0.040255,...,215,-1,28,-1,-1,-1,-1,423,-1,0
702083,0,202203,-0.357336,-0.120629,1.448792,-0.494663,-0.055048,-0.18704,-0.03591,-0.040255,...,215,0,28,3,-1,1,-1,1474,1,0
702084,0,202201,-0.357336,-0.120629,-0.764699,-0.546524,-0.055048,-0.18704,-0.03591,-0.040255,...,268,-1,28,-1,-1,-1,-1,423,-1,0


# Обучение модели

In [154]:
# Задаем объясняемую и объясняющие переменные
features = data_fill.drop(columns=['target'])
target = data_fill['target']

In [155]:
# Оределеяем константу для random_state
STATE = 442

# Создаем метрику качества для GridSearch
roc_auc_scorer = make_scorer(roc_auc_score, greater_is_better=True,
                             needs_threshold=True)


# Определяем модель
model = lgb.LGBMClassifier(random_state = STATE)

In [156]:
%%time
params = {
     'num_iterations':[300],
     'max_depth': [10],
     'reg_alpha':[0.01],
     'num_leaves': [5],
     'num_threads': [6]}
# Настраиваем GridSearch
grid_lgbm = GridSearchCV(model, 
                         params, 
                         cv = 4, 
                         scoring = roc_auc_scorer, 
                         n_jobs = -1)

# Находим модель с лучшим значением метрики
grid_lgbm.fit(features[features.columns.tolist()], target)


print(f'ROC-AUC для лучшей модели бустинга составило: '
      f'{-grid_lgbm.best_score_:.6}')
print(f'Гиперпараметры лучшей модели: {grid_lgbm.best_params_}')

H:\Anaconda\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


ROC-AUC для лучшей модели бустинга составило: -0.733815
Гиперпараметры лучшей модели: {'max_depth': 10, 'num_iterations': 300, 'num_leaves': 5, 'num_threads': 6, 'reg_alpha': 0.01}
Wall time: 2min 5s


In [157]:
# Достаем значимость объясняющих переменных
importances = grid_lgbm.best_estimator_.feature_importances_

# Создаем датафрейм
data_tuples = list(zip(features.columns.tolist(), importances.tolist()))
feat_imp_df = pd.DataFrame(data_tuples, columns=['feature','importance'])
feat_imp_df.sort_values(by = 'importance', ascending = False)

,feature,importance
0,period,29
152,materials_details_13_1_ctg,24
74,traffic_details_49_6_std,24
145,tariff_plans_18_1_ctg,22
144,materials_details_0_1_ctg,22
...,...,...
9,spas_symptoms_agr_114_6_std,1
131,campaigns_352_1_sum,1
88,spas_symptoms_agr_223_6_sum,1
27,spas_symptoms_int_114_1_cnt,1


## Тестирование модели

In [158]:
# Считываем тестовые данные
test = pd.read_parquet('features_oot.parquet', engine='pyarrow')

# Считываем объясняющие переменные
features_test = test[features.columns.tolist()]

# Можно сразу считать фичи из паркета

In [159]:
# Прогнозируем значение
predict = grid_lgbm.predict_proba(features_test)

In [160]:
# Подготавливаем данные для экспорта в csv
predict_df = pd.DataFrame(predict).reset_index()
predict_df = predict_df.drop(columns = 0).rename(columns = {'index': 'id', 1: 'target'})
predict_df

,id,target
0,0,0.006275
1,1,0.003728
2,2,0.011494
3,3,0.004412
4,4,0.011687
...,...,...
60656,60656,0.004887
60657,60657,0.004500
60658,60658,0.032420
60659,60659,0.014658


In [161]:
# Экспортируем файл
predict_df.to_csv (r'C:\Users\anato\Desktop\ds\Хакатон R1\3nd_chanse_full_most_important_with_drop.csv', index= False )